In [ ]:
import glob
import sys
import os
import time
sys.path.append('../tests/')

In [ ]:
import xarray as xr
import pandas as pd
import cfgrib

import cdscommon

In [ ]:
TEST_FILES = {
    #ERA5
    'era5-single-levels-reanalysis': [
        'reanalysis-era5-single-levels',
        {
            'variable': '2m_temperature',
            'product_type': 'reanalysis',
            'year': '2017',
            'month': '01',
            'day': ['01', '02'],
            'time': ['00:00', '12:00'],
            'grid': ['3', '3'],
            'format': 'grib',
        },
        191,
    ],
    'era5-single-levels-ensemble_members': [
        'reanalysis-era5-single-levels',
        {
            'variable': '2m_temperature',
            'product_type': 'ensemble_members',
            'year': '2017',
            'month': '01',
            'day': ['01', '02'],
            'time': ['00:00', '12:00'],
            'grid': ['3', '3'],
            'format': 'grib',
        },
        192,
    ],
    'era5-single-levels-ensemble_mean': [
        'reanalysis-era5-single-levels',
        {
            'variable': '2m_temperature',
            'product_type': 'ensemble_mean',
            'year': '2017',
            'month': '01',
            'day': ['01', '02'],
            'time': ['00:00', '12:00'],
            'grid': ['3', '3'],
            'format': 'grib',
        },
        193,
    ],
    'era5-single-levels-ensemble_spread': [
        'reanalysis-era5-single-levels',
        {
            'variable': '2m_temperature',
            'product_type': 'ensemble_spread',
            'year': '2017',
            'month': '01',
            'day': ['01', '02'],
            'time': ['00:00', '12:00'],
            'grid': ['3', '3'],
            'format': 'grib',
        },
        193,
    ],
    'era5-single-levels-reanalysis-area': [
        'reanalysis-era5-single-levels',
        {
            'variable': '2m_temperature',
            'product_type': 'reanalysis',
            'year': '2017',
            'month': '01',
            'day': ['01', '02'],
            'time': ['00:00', '12:00'],
            'area': ['35.5', '6.5', '47.', '19.'],
            'format': 'grib',
        },
        191,
    ],
    'era5-pressure-levels-reanalysis': [
        'reanalysis-era5-pressure-levels',
        {
            'variable': 'temperature',
            'pressure_level': ['500', '850'],
            'product_type': 'reanalysis',
            'year': '2017',
            'month': '01',
            'day': ['01', '02'],
            'time': ['00:00', '12:00'],
            'grid': ['3', '3'],
            'format': 'grib',
        },
        191,
    ],
    'era5-pressure-levels-ensemble_members': [
        'reanalysis-era5-pressure-levels',
        {
            'variable': 'temperature',
            'pressure_level': ['500', '850'],
            'product_type': 'ensemble_members',
            'year': '2017',
            'month': '01',
            'day': ['01', '02'],
            'time': ['00:00', '12:00'],
            'grid': ['3', '3'],
            'format': 'grib',
        },
        192,
    ],
    'era5-pressure-levels-ensemble_mean': [
        'reanalysis-era5-pressure-levels',
        {
            'product_type': 'ensemble_mean',
            'variable': 'temperature',
            'pressure_level': '1',
            'year': '2017',
            'month': '01',
            'day': '01',
            'time': ['00:00', '03:00'],
        },
        193,
    ],
    'era5-pressure-levels-ensemble_spread': [
        'reanalysis-era5-pressure-levels',
        {
            'product_type': 'ensemble_spread',
            'variable': 'temperature',
            'pressure_level': '1',
            'year': '2017',
            'month': '01',
            'day': '01',
            'time': ['00:00', '03:00'],
        },
        193,
    ],
    #ERA5 MONTHLY MEANS
    'era5-single-levels-monthly-means-monthly_averaged_reanalysis': [
        'reanalysis-era5-single-levels-monthly-means',
        {
            'format': 'grib',
            'product_type': 'monthly_averaged_reanalysis',
            'variable': '2m_temperature',
            'year': '2017',
            'month': ['01', '02'],
            'time': '00:00',
        },
        190,
    ],
    'era5-single-levels-monthly-means-monthly_averaged_reanalysis_by_hour_of_day': [
        'reanalysis-era5-single-levels-monthly-means',
        {
            'format': 'grib',
            'product_type': 'monthly_averaged_reanalysis_by_hour_of_day',
            'variable': '2m_temperature',
            'year': '2017',
            'month': ['01', '02'],
            'time': '00:00',
        },
        190,
    ],
    'era5-single-levels-monthly-means-monthly_averaged_ensemble_members': [
        'reanalysis-era5-single-levels-monthly-means',
        {
            'format': 'grib',
            'product_type': 'monthly_averaged_ensemble_members',
            'variable': '2m_temperature',
            'year': '2017',
            'month': ['01', '02'],
            'time': '00:00',
        },
        192,
    ],
    'era5-single-levels-monthly-means-monthly_averaged_ensemble_members_by_hour_of_day': [
        'reanalysis-era5-single-levels-monthly-means',
        {
            'format': 'grib',
            'product_type': 'monthly_averaged_ensemble_members_by_hour_of_day',
            'variable': '2m_temperature',
            'year': '2017',
            'month': ['01', '02'],
            'time': '00:00',
        },
        192,
    ],
    'era5-pressure-levels-monthly-means-monthly_averaged_reanalysis': [
        'reanalysis-era5-pressure-levels-monthly-means',
        {
            'product_type': 'monthly_averaged_reanalysis',
            'variable': 'divergence',
            'pressure_level': '1',
            'year': '2017',
            'month': ['01', '02'],
            'time': '00:00',
        },
        190,
    ],
    'era5-pressure-levels-monthly-means-monthly_averaged_reanalysis_by_hour_of_day': [
        'reanalysis-era5-pressure-levels-monthly-means',
        {
            'product_type': 'monthly_averaged_reanalysis_by_hour_of_day',
            'variable': 'divergence',
            'pressure_level': '1',
            'year': '2017',
            'month': ['01', '02'],
            'time': '00:00',
        },
        190,
    ],
    'era5-pressure-levels-monthly-means-monthly_averaged_ensemble_members': [
        'reanalysis-era5-pressure-levels-monthly-means',
        {
            'product_type': 'monthly_averaged_ensemble_members',
            'variable': 'divergence',
            'pressure_level': '1',
            'year': '2017',
            'month': ['01', '02'],
            'time': '00:00',
        },
        192,
    ],
    'era5-pressure-levels-monthly-means-monthly_averaged_ensemble_members_by_hour_of_day': [
        'reanalysis-era5-pressure-levels-monthly-means',
        {
            'product_type': 'monthly_averaged_ensemble_members_by_hour_of_day',
            'variable': 'divergence',
            'pressure_level': '1',
            'year': '2017',
            'month': ['01', '02'],
            'time': '00:00',
        },
        192,
    ],
    #ERA5-Land
        'era5-land-reanalysis': [
        'reanalysis-era5-land',
        {
            'format': 'grib',
            'variable': '2m_dewpoint_temperature',
            'year': '2017',
            'month': '01',
            'day': '01',
            'time': ['00:00', '01:00'],
        },
        197,
    ],
    'era5-land-monthly-means-monthly_averaged_reanalysis': [
        'reanalysis-era5-land-monthly-means',
        {
            'format': 'grib',
            'product_type': 'monthly_averaged_reanalysis',
            'variable': '2m_dewpoint_temperature',
            'year': '2017',
            'month': ['01', '02'],
            'time': '00:00',
        },
        203,
    ],
    'era5-land-monthly-means-monthly_averaged_reanalysis_by_hour_of_day': [
        'reanalysis-era5-land-monthly-means',
        {
            'format': 'grib',
            'product_type': 'monthly_averaged_reanalysis_by_hour_of_day',
            'variable': '2m_dewpoint_temperature',
            'year': '2017',
            'month': '01',
            'time': ['00:00', '01:00'],
        },
        203,
    ],
    #Seasonal forecast
    'seasonal-original-single-levels-ecmwf': [
        'seasonal-original-single-levels',
        {
            'originating_centre': 'ecmwf',
            'variable': 'maximum_2m_temperature_in_the_last_24_hours',
            'year': '2018',
            'month': ['04', '05'],
            'day': [
                '01',
                '02',
                '03',
                '04',
                '05',
                '06',
                '07',
                '08',
                '09',
                '10',
                '11',
                '12',
                '13',
                '14',
                '15',
                '16',
                '17',
                '18',
                '19',
                '20',
                '21',
                '22',
                '23',
                '24',
                '25',
                '26',
                '27',
                '28',
                '29',
                '30',
                '31',
            ],
            'leadtime_hour': ['24', '48'],
            'grid': ['3', '3'],
            'format': 'grib',
        },
        192,
    ],
    'seasonal-original-pressure-levels-ecmwf': [
        'seasonal-original-pressure-levels',
        {
            'originating_centre': 'ecmwf',
            'variable': 'temperature',
            'pressure_level': ['500', '850'],
            'year': '2018',
            'month': ['04', '05'],
            'day': [
                '01',
                '02',
                '03',
                '04',
                '05',
                '06',
                '07',
                '08',
                '09',
                '10',
                '11',
                '12',
                '13',
                '14',
                '15',
                '16',
                '17',
                '18',
                '19',
                '20',
                '21',
                '22',
                '23',
                '24',
                '25',
                '26',
                '27',
                '28',
                '29',
                '30',
                '31',
            ],
            'leadtime_hour': ['24', '48'],
            'grid': ['3', '3'],
            'format': 'grib',
        },
        192,
    ],
    'seasonal-postprocessed-single-levels-ecmwf': [
        'seasonal-postprocessed-single-levels',
        {
            'originating_centre': 'ecmwf',
            'variable': 'maximum_2m_temperature_in_the_last_24_hours_anomaly',
            'product_type': 'monthly_mean',
            'year': '2018',
            'month': ['04', '05'],
            'leadtime_month': ['1', '2'],
            'grid': ['3', '3'],
            'format': 'grib',
        },
        211,
    ],
    'seasonal-monthly-single-levels-monthly_mean-ecmwf': [
        'seasonal-monthly-single-levels',
        {
            'originating_centre': 'ecmwf',
            'variable': 'maximum_2m_temperature_in_the_last_24_hours',
            'product_type': 'monthly_mean',
            'year': '2018',
            'month': ['04', '05'],
            'leadtime_month': ['1', '2'],
            'grid': ['3', '3'],
            'format': 'grib',
        },
        211,
    ],
    'seasonal-monthly-single-levels-ensemble_mean-ecmwf': [
        'seasonal-monthly-single-levels',
        {
            'originating_centre': 'ecmwf',
            'variable': 'maximum_2m_temperature_in_the_last_24_hours',
            'product_type': 'ensemble_mean',
            'year': '2018',
            'month': ['04', '05'],
            'leadtime_month': ['1', '2'],
            'grid': ['3', '3'],
            'format': 'grib',
        },
        211,
    ],
    'seasonal-monthly-single-levels-hindcast_climate_mean-ecmwf': [
        'seasonal-monthly-single-levels',
        {
            'originating_centre': 'ecmwf',
            'variable': 'maximum_2m_temperature_in_the_last_24_hours',
            'product_type': 'hindcast_climate_mean',
            'year': '2018',
            'month': ['04', '05'],
            'leadtime_month': ['1', '2'],
            'grid': ['3', '3'],
            'format': 'grib',
        },
        211,
    ],
    'seasonal-original-single-levels-meteo_france': [
        'seasonal-original-single-levels',
        {
            'originating_centre': 'meteo_france',
            'variable': 'maximum_2m_temperature_in_the_last_24_hours',
            'year': '2018',
            'month': ['04', '05'],
            'day': [
                '01',
                '02',
                '03',
                '04',
                '05',
                '06',
                '07',
                '08',
                '09',
                '10',
                '11',
                '12',
                '13',
                '14',
                '15',
                '16',
                '17',
                '18',
                '19',
                '20',
                '21',
                '22',
                '23',
                '24',
                '25',
                '26',
                '27',
                '28',
                '29',
                '30',
                '31',
            ],
            'leadtime_hour': ['24', '48'],
            'grid': ['3', '3'],
            'format': 'grib',
        },
        193,
    ],
    'seasonal-original-pressure-levels-meteo_france': [
        'seasonal-original-pressure-levels',
        {
            'originating_centre': 'meteo_france',
            'variable': 'temperature',
            'pressure_level': ['500', '850'],
            'year': '2018',
            'month': ['04', '05'],
            'day': [
                '01',
                '02',
                '03',
                '04',
                '05',
                '06',
                '07',
                '08',
                '09',
                '10',
                '11',
                '12',
                '13',
                '14',
                '15',
                '16',
                '17',
                '18',
                '19',
                '20',
                '21',
                '22',
                '23',
                '24',
                '25',
                '26',
                '27',
                '28',
                '29',
                '30',
                '31',
            ],
            'leadtime_hour': ['24', '48'],
            'grid': ['3', '3'],
            'format': 'grib',
        },
        193,
    ],
    'seasonal-postprocessed-single-levels-meteo_france': [
        'seasonal-postprocessed-single-levels',
        {
            'originating_centre': 'meteo_france',
            'variable': 'maximum_2m_temperature_in_the_last_24_hours_anomaly',
            'product_type': 'monthly_mean',
            'year': '2018',
            'month': ['04', '05'],
            'leadtime_month': ['1', '2'],
            'grid': ['3', '3'],
            'format': 'grib',
        },
        212,
    ],
    'seasonal-monthly-single-levels-monthly_mean-meteo_france': [
        'seasonal-monthly-single-levels',
        {
            'originating_centre': 'meteo_france',
            'variable': 'maximum_2m_temperature_in_the_last_24_hours',
            'product_type': 'monthly_mean',
            'year': '2018',
            'month': ['04', '05'],
            'leadtime_month': ['1', '2'],
            'grid': ['3', '3'],
            'format': 'grib',
        },
        212,
    ],
    'seasonal-monthly-single-levels-ensemble_mean-meteo_france': [
        'seasonal-monthly-single-levels',
        {
            'originating_centre': 'meteo_france',
            'variable': 'maximum_2m_temperature_in_the_last_24_hours',
            'product_type': 'ensemble_mean',
            'year': '2018',
            'month': ['04', '05'],
            'leadtime_month': ['1', '2'],
            'grid': ['3', '3'],
            'format': 'grib',
        },
        212,
    ],
    'seasonal-monthly-single-levels-hindcast_climate_mean-meteo_france': [
        'seasonal-monthly-single-levels',
        {
            'originating_centre': 'meteo_france',
            'variable': 'maximum_2m_temperature_in_the_last_24_hours',
            'product_type': 'hindcast_climate_mean',
            'year': '2018',
            'month': ['04', '05'],
            'leadtime_month': ['1', '2'],
            'grid': ['3', '3'],
            'format': 'grib',
        },
        212,
    ],
    'seasonal-original-single-levels-ukmo': [
        'seasonal-original-single-levels',
        {
            'originating_centre': 'ukmo',
            'variable': 'maximum_2m_temperature_in_the_last_24_hours',
            'year': '2018',
            'month': ['04', '05'],
            'day': [
                '01',
                '02',
                '03',
                '04',
                '05',
                '06',
                '07',
                '08',
                '09',
                '10',
                '11',
                '12',
                '13',
                '14',
                '15',
                '16',
                '17',
                '18',
                '19',
                '20',
                '21',
                '22',
                '23',
                '24',
                '25',
                '26',
                '27',
                '28',
                '29',
                '30',
                '31',
            ],
            'leadtime_hour': ['24', '48'],
            'grid': ['3', '3'],
            'format': 'grib',
        },
        193,
    ],
    'seasonal-original-pressure-levels-ukmo': [
        'seasonal-original-pressure-levels',
        {
            'originating_centre': 'ukmo',
            'variable': 'temperature',
            'pressure_level': ['500', '850'],
            'year': '2018',
            'month': ['04', '05'],
            'day': [
                '01',
                '02',
                '03',
                '04',
                '05',
                '06',
                '07',
                '08',
                '09',
                '10',
                '11',
                '12',
                '13',
                '14',
                '15',
                '16',
                '17',
                '18',
                '19',
                '20',
                '21',
                '22',
                '23',
                '24',
                '25',
                '26',
                '27',
                '28',
                '29',
                '30',
                '31',
            ],
            'leadtime_hour': ['24', '48'],
            'grid': ['3', '3'],
            'format': 'grib',
        },
        193,
    ],
    'seasonal-postprocessed-single-levels-ukmo': [
        'seasonal-postprocessed-single-levels',
        {
            'originating_centre': 'ukmo',
            'variable': 'maximum_2m_temperature_in_the_last_24_hours_anomaly',
            'product_type': 'monthly_mean',
            'year': '2018',
            'month': ['04', '05'],
            'leadtime_month': ['1', '2'],
            'grid': ['3', '3'],
            'format': 'grib',
        },
        212,
    ],
    'seasonal-monthly-single-levels-monthly_mean-ukmo': [
        'seasonal-monthly-single-levels',
        {
            'originating_centre': 'ukmo',
            'variable': 'maximum_2m_temperature_in_the_last_24_hours',
            'product_type': 'monthly_mean',
            'year': '2018',
            'month': ['04', '05'],
            'leadtime_month': ['1', '2'],
            'grid': ['3', '3'],
            'format': 'grib',
        },
        212,
    ],
    'seasonal-monthly-single-levels-ensemble_mean-ukmo': [
        'seasonal-monthly-single-levels',
        {
            'originating_centre': 'ukmo',
            'variable': 'maximum_2m_temperature_in_the_last_24_hours',
            'product_type': 'ensemble_mean',
            'year': '2018',
            'month': ['04', '05'],
            'leadtime_month': ['1', '2'],
            'grid': ['3', '3'],
            'format': 'grib',
        },
        212,
    ],
    'seasonal-monthly-single-levels-hindcast_climate_mean-ukmo': [
        'seasonal-monthly-single-levels',
        {
            'originating_centre': 'ukmo',
            'variable': 'maximum_2m_temperature_in_the_last_24_hours',
            'product_type': 'hindcast_climate_mean',
            'year': '2018',
            'month': ['04', '05'],
            'leadtime_month': ['1', '2'],
            'grid': ['3', '3'],
            'format': 'grib',
        },
        212,
    ],
    #UERRA
    'uerra-europe-height-levels-reanalysis': [
        'reanalysis-uerra-europe-height-levels',
        {
            'format': 'grib',
            'variable': 'pressure',
            'height_level': '15_m',
            'year': '2017',
            'month': '01',
            'day': '01',
            'time': ['00:00', '06:00'],
        },
        231,
    ],
    'uerra-europe-single-levels-reanalysis-mescan_surfex': [
        'reanalysis-uerra-europe-single-levels',
        {
            'format': 'grib',
            'origin': 'mescan_surfex',
            'variable': '10m_wind_direction',
            'year': '2017',
            'month': '01',
            'day': '01',
            'time': ['00:00', '06:00'],
        },
        231,
    ],
    'uerra-europe-single-levels-reanalysis-uerra_harmonie': [
        'reanalysis-uerra-europe-single-levels',
        {
            'format': 'grib',
            'origin': 'uerra_harmonie',
            'variable': '10m_wind_direction',
            'year': '2017',
            'month': '01',
            'day': '01',
            'time': ['00:00', '06:00'],
        },
        230,
    ],
    'uerra-europe-pressure-levels-reanalysis': [
        'reanalysis-uerra-europe-pressure-levels',
        {
            'format': 'grib',
            'variable': 'geopotential',
            'pressure_level': '10',
            'year': '2017',
            'month': '01',
            'day': '01',
            'time': ['00:00', '06:00'],
        },
        230,
    ],
    'uerra-europe-soil-levels-reanalysis-mescan_surfex': [
        'reanalysis-uerra-europe-soil-levels',
        {
            'format': 'grib',
            'origin': 'mescan_surfex',
            'variable': 'volumetric_transpiration_stress_onset',
            'soil_level': '1',
            'year': '2017',
            'month': '01',
            'day': '01',
            'time': ['00:00', '06:00'],
        },
        230,
    ],
    'uerra-europe-soil-levels-reanalysis-uerra_harmonie': [
        'reanalysis-uerra-europe-soil-levels',
        {
            'format': 'grib',
            'origin': 'uerra_harmonie',
            'variable': 'soil_temperature',
            'soil_level': '1',
            'year': '2017',
            'month': '01',
            'day': '01',
            'time': ['00:00', '06:00'],
        },
        231,
    ],
    #EFAS
    'efas-forecast-control_forecast-surface_level': [
        'efas-forecast',
        {
            'format': 'grib',
            'origin': 'ecmwf',
            'type': 'control_forecast',
            'variable': 'river_discharge_in_the_last_24_hours',
            'model_levels': 'surface_level',
            'year': '2018',
            'month': '10',
            'day': '11',
            'time': '00:00',
            'step': ['24', '48'],
        },
        271,
    ],
    'efas-forecast-ensemble_perturbed_forecasts-surface_level': [
        'efas-forecast',
        {
            'origin': 'ecmwf',
            'type': 'ensemble_perturbed_forecasts',
            'variable': 'river_discharge_in_the_last_24_hours',
            'model_levels': 'surface_level',
            'year': '2018',
            'month': '10',
            'day': '11',
            'time': '00:00',
            'step': ['24', '48'],
            'format': 'grib',
        },
        271,
    ],
    'efas-forecast-high_resolution_forecast-surface_level': [
        'efas-forecast',
        {
            'origin': 'ecmwf',
            'type': 'high_resolution_forecast',
            'variable': 'river_discharge_in_the_last_6_hours',
            'model_levels': 'surface_level',
            'year': '2018',
            'month': '10',
            'day': '11',
            'time': '00:00',
            'step': ['24', '48'],
            'format': 'grib',
        },
        268,
    ],
    'efas-forecast-control_forecast-soil_levels': [
        'efas-forecast',
        {
            'origin': 'ecmwf',
            'type': 'control_forecast',
            'variable': 'soil_depth',
            'model_levels': 'soil_levels',
            'year': '2019',
            'month': '01',
            'day': '30',
            'time': '00:00',
            'step': ['0', '24'],
            'format': 'grib',
            'soil_level': '1',
        },
        256,
    ],
    'efas-historical-version_4_0-surface_level': [
        'efas-historical',
        {
            'format': 'grib',
            'simulation_version': 'version_4_0',
            'variable': 'snow_depth_water_equivalent',
            'model_levels': 'surface_level',
            'hyear': '2017',
            'hmonth': 'january',
            'hday': ['01', '02'],
            'time': '06:00',
        },
        255,
    ],
    'efas-historical-version_3_5-surface_level': [
        'efas-historical',
        {
            'format': 'grib',
            'simulation_version': 'version_3_5',
            'variable': 'river_discharge',
            'model_levels': 'surface_level',
            'hyear': '2017',
            'hmonth': 'january',
            'hday': ['01', '02'],
            'time': '06:00'
        },
        269,
    ],
    'efas-historical-version_3_0-surface_level': [
        'efas-historical',
        {
            'format': 'grib',
            'simulation_version': 'version_3_0',
            'variable': 'river_discharge',
            'model_levels': 'surface_level',
            'hyear': '2017',
            'hmonth': 'january',
            'hday': ['01', '02'],
            'time': '06:00'
        },
        268,
    ],
    'efas-historical-version_2_0-surface_level': [
        'efas-historical',
        {
            'format': 'grib',
            'simulation_version': 'version_2_0',
            'variable': 'river_discharge',
            'model_levels': 'surface_level',
            'hyear': '2017',
            'hmonth': 'january',
            'hday': ['01', '02'],
            'time': '06:00'
        },
        268,
    ],
    'efas-historical-version_4_0-soil_level': [
        'efas-historical',
        {
            'format': 'grib',
            'simulation_version': 'version_4_0',
            'variable': 'soil_depth',
            'model_levels': 'soil_levels',
            'hyear': '2017',
            'hmonth': 'january',
            'hday': ['01', '02'],
            'time': '06:00',
            'soil_level': '1',
        },
        254,
    ],
    'efas-historical-version_3_5-soil_level': [
        'efas-historical',
        {
            'format': 'grib',
            'simulation_version': 'version_3_5',
            'variable': 'soil_depth',
            'model_levels': 'soil_levels',
            'hyear': '2017',
            'hmonth': 'january',
            'hday': ['01', '02'],
            'soil_level': '1',
            'time': '06:00'
        },
        254,
    ],
    #CEMS-GLOFAS
    'cems-glofas-forecast-ensemble_perturbed_forecasts': [
        'cems-glofas-forecast',
        {
            'format': 'grib',
            'type': 'ensemble_perturbed_forecasts',
            'year': '2020',
            'month': '01',
            'day': '01',
            'step': ['24', '48'],
        },
        288,
    ],
    'cems-glofas-forecast-control_forecast': [
        'cems-glofas-forecast',
        {
            'format': 'grib',
            'type': 'control_forecast',
            'year': '2020',
            'month': '01',
            'day': '01',
            'step': ['24', '48'],
        },
        288,
    ],
}

In [ ]:
def eccodes_to_xarray(input_file, output_file):
    os.system(f'grib_to_netcdf -o {output_file} {input_file}')
    
def cfgrib_to_xarray(input_file, output_file):
    ds = xr.open_dataset(input_file, engine='cfgrib')
    ds.to_netcdf(output_file)
    ds.close()

In [ ]:
# dataframe
df = pd.DataFrame(columns=['file', 'size', 'eccodes_time', 'cfgrib_time'])

for test_file in TEST_FILES.keys():
    
    dataset, request, key_count = TEST_FILES[test_file]
    
    print(dataset, request)
    input_path = cdscommon.ensure_data(dataset, request, name='cds-' + test_file + '-{uuid}{ext}')
    
    size = os.stat(input_path).st_size
    
    output_path_eccodes = input_path.replace('.grib', '_eccodes.nc')
    output_path_cfgrib = input_path.replace('.grib', '_cfgrib.nc')

    start_time = time.time()
    eccodes_time = 'NA'
    eccodes_to_xarray(input_path, output_path_eccodes)
    if os.path.isfile(output_path_eccodes):
        eccodes_time = time.time() - start_time
    print(eccodes_time)

    start_time = time.time()
    cfgrib_time = 'NA'
    cfgrib_to_xarray(input_path, output_path_cfgrib)
    if os.path.isfile(output_path_cfgrib):
        cfgrib_time = time.time() - start_time
    print(cfgrib_time)

    df = df.append(
        {
            'file': test_file, 
            'size': size, 'eccodes_time': eccodes_time, 
            'cfgrib_time': cfgrib_time
        }, 
        ignore_index=True
    )
    
# remove all .idx files
idx_files = glob.glob('../tests/sample-data/*.idx')
for idx_file in idx_files:
    os.remove(idx_file)

In [ ]:
df